In [ ]:
#Import the necessary libraries

import pandas as pd
import psycopg2

#Connect to Databases DVD in postgresql
conn = psycopg2.connect("dbname=dvd user=postgres password=franklampard8")

#Create queries for several tables
cur = conn.cursor()
sql = "select * from payment"
sql2 = "select * from rental"
sql3 = "select* from inventory"
sql4 = "select customer_id, address_id from customer"
sql5 = "select address_id, city_id from address"
sql6 = "select city_id, city, country_id from city"
sql7 = "select country_id, country from country"

#Create dataframe
dfpay = pd.read_sql_query(sql, con=conn)
dfrent = pd.read_sql_query(sql2, con=conn)
dfinven = pd.read_sql_query(sql3, con=conn)
dfcust = pd.read_sql_query(sql4, con=conn)
dfaddr = pd.read_sql_query(sql5, con=conn)
dfcity = pd.read_sql_query(sql6, con=conn)
dfcountry = pd.read_sql_query(sql7, con=conn)

#Merge several dataframes (dfrent, dfinven, and dfpay) into variable df1
df1 = dfrent[['rental_id', 'inventory_id']].merge(dfinven[['film_id','inventory_id']], on='inventory_id').merge(dfpay, 
                                                                                                                on='rental_id')

#Merge df1 with dfcust, dfaddr, dfcity, and dfcountry into variable df2
df2 = df1.merge(dfcust, on='customer_id').merge(dfaddr, on='address_id').merge(dfcity, on='city_id').merge(dfcountry, 
                                                                                                           on='country_id')

#Extract year-month-date from payment_date column into payment_date_string column
df2['payment_date_string'] = df2.payment_date.dt.strftime('%Y-%m-%d')

#Group df2 by 'film_id','payment_date_string','country_id','city_id' and aggregate 'film_id' by count
df3 = df2.groupby(['film_id','payment_date_string','country_id','city_id']).agg({'film_id':'count'}).rename(
    columns={'film_id':'Number_film_purchased'}).reset_index()

#change payment_date_string to datetime and change the column name into paymentdate
import datetime

df3['payment_date_string'] = pd.to_datetime(df3['payment_date_string'])
df3.rename(columns={'payment_date_string':'paymentdate'}, inplace=True)

#reset_index
df4 = df3.reset_index()
df4.rename(columns={'index':'purchase_id'}, inplace=True)
df4.purchase_id = df4.purchase_id + 1

#Export dataframe (df4) to csv
df4.to_csv('D:\dickys\Work\IYKRA\ProficiencyTest\dfilmpurchased.csv', index=False)

#Create filmpurchased table
def create_table():
    conn = psycopg2.connect("dbname=dvd user=postgres password=franklampard8")

    cur = conn.cursor()
    
    cur.execute("""CREATE TABLE filmpurchased
                (purchase_id serial not null,
                film_id int2 not null, 
                paymentdate DATE not null,
                country_id int2, 
                city_id int2, 
                Number_film_purchased int2 not null,
                constraint filmpurchased_pkey PRIMARY KEY (purchase_id),
                constraint filmpurchased_city_fkey foreign key (city_id) references city(city_id),
                constraint filmpurchased_country_fkey foreign key (country_id) references country(country_id))""")
    
    cur.execute("""SELECT 
               table_name, 
               column_name, 
               data_type 
               FROM 
               information_schema.columns
               WHERE 
               table_name = 'film_purchase'""")
    
    conn.commit()
    
    return cur.fetchall()

create_table()

#import data from csv to filmpurchased table
def import_table():
    conn = psycopg2.connect("dbname=dvd user=postgres password=franklampard8")

    cur = conn.cursor()
    
    cur.execute("""COPY filmpurchased
                FROM 'D:\dickys\Work\IYKRA\ProficiencyTest\dfilmpurchased.csv'
                DELIMITER ','
                CSV HEADER""")
    
    cur.execute("""SELECT *
               FROM filmpurchased""")
    
    conn.commit()
    
    return cur.fetchall()

import_table()